# Web Scraping
Según el procedimiento en https://www.datacamp.com/community/tutorials/web-scraping-using-python

### Pendientes: 
- ¿Cómo leer y fusionar diferentes campos en un solo csv?
- Añadir lista de HTML para ingreso a cada una de las notas  

In [57]:
#Importes necesarios
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime

In [61]:
#Establecer función para el cambio de fechas
def fecha_variable(start_date):
    fecha = start_date.strftime("%Y-%m-%d")
    url = "https://www.elsiglodetorreon.com.mx/archivo/?d=" + fecha
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    for div in soup.find_all('div', {'class': 'col-xs-9'}):
         for atag in div.find_all('a'):
                    f = open('%s.txt' % fecha, "a", encoding='utf-8')
                    f.write("\""+atag.text+"\"\n")
                    f.close()

In [63]:
#Variabilización de las fechas y ejecución de la función

start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 3, 4)
delta = datetime.timedelta(days=1)

while start_date <= end_date:
    fecha_variable(start_date)
    start_date += delta

In [52]:
#Pendiente -> Leer el archivo .txt como .csv para importar a df
#read_file = pd.read_csv (r'2021-02-01.txt')
#read_file.columns = ['Titular']
#read_file.to_csv (r'2021-02-01.csv', index=None)